# Donkey Colab with TPU

Donkey Colab with TPU.

# DonkeyレポジトリをGit clone

In [0]:
!git clone -b dev https://github.com/autorope/donkeycar.git donkey

Cloning into 'donkey'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 6688 (delta 0), reused 0 (delta 0), pack-reused 6687
Receiving objects: 100% (6688/6688), 41.53 MiB | 35.95 MiB/s, done.
Resolving deltas: 100% (3852/3852), done.


# インストール

In [0]:
!pip3 install -e donkey

Obtaining file:///content/donkey
    100% |████████████████████████████████| 51kB 2.5MB/s 
    100% |████████████████████████████████| 225kB 8.7MB/s 
    100% |████████████████████████████████| 122kB 36.1MB/s 
    100% |████████████████████████████████| 194kB 30.5MB/s 
  Running setup.py develop for donkeycar


# Projectの作成

In [0]:
!donkey createcar --template donkey2  /content/mycar

using donkey version: 2.5.7 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: donkey2
Copying car config defaults. Adjust these before starting your car.
Donkey setup complete.


# datas.zipのアップロード

**ファイル-アップロード**を選択し、PCからdatas.zipをColabにアップロードします。

In [0]:
!mv datas.zip /content/mycar/data/

In [0]:
%cd /content/mycar/data/

!unzip /content/mycar/data/datas.zip

/content/mycar/data
Archive:  /content/mycar/data/datas.zip
   creating: tub_01_19-01-31/
  inflating: tub_01_19-01-31/21_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/record_73.json  
  inflating: tub_01_19-01-31/15_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/34_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/record_44.json  
  inflating: tub_01_19-01-31/59_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/14_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/record_65.json  
  inflating: tub_01_19-01-31/76_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/30_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/90_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/75_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/69_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/record_14.json  
  inflating: tub_01_19-01-31/record_49.json  
  inflating: tub_01_19-01-31/84_cam-image_array_.jpg  
  inflating: tub_01_19-01-31/record_80.json  
  inflating: tub_01_19-01-31/

# 学習用コードの呼び出し

学習する前に、学習用コードを読み込みます。


In [0]:
""""

keras.py

functions to run and train autopilots using keras

"""
import os
import tensorflow as tf
from tensorflow.contrib.tpu.python.tpu import keras_support
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras.layers import Dropout, Flatten, Dense
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

class KerasPilot:

    def load(self, model_path):
        #with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        self.model = load_model(model_path)

    def shutdown(self):
        pass

    def train(self, train_gen, val_gen,
              saved_model_path, epochs=100, steps=100, train_split=0.8,
              verbose=1, min_delta=.0005, patience=5, use_early_stop=True):
        """
        train_gen: generator that yields an array of images an array of

        """

        # checkpoint to save model after each epoch
        save_best = ModelCheckpoint(saved_model_path,
                                    monitor='val_loss',
                                    verbose=verbose,
                                    save_best_only=True,
                                    mode='min')

        # stop training if the validation error stops improving.
        early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=min_delta,
                                   patience=patience,
                                   verbose=verbose,
                                   mode='auto')

        callbacks_list = [save_best]

        if use_early_stop:
            callbacks_list.append(early_stop)

        hist = self.model.fit_generator(
            train_gen,
            steps_per_epoch=steps,
            epochs=epochs,
            verbose=1,
            validation_data=val_gen,
            callbacks=callbacks_list,
            validation_steps=steps * (1.0 - train_split) / train_split)
        return hist


class KerasLinear(KerasPilot):
    def __init__(self, model=None, num_outputs=None, *args, **kwargs):
        super(KerasLinear, self).__init__(*args, **kwargs)
        if model:
            self.model = model
        elif num_outputs is not None:
            self.model = default_linear()
        else:
            self.model = default_linear()

    def run(self, img_arr):
        img_arr = img_arr.reshape((1,) + img_arr.shape)
        outputs = self.model.predict(img_arr)
        # print(len(outputs), outputs)
        steering = outputs[0]
        throttle = outputs[1]
        return steering[0][0], throttle[0][0]


def default_linear():
    img_in = Input(shape=(120, 160, 3), name='img_in')
    x = img_in

    # Convolution2D class name is an alias for Conv2D
    x = Convolution2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=32, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)

    x = Flatten(name='flattened')(x)
    x = Dense(units=100, activation='linear')(x)
    x = Dropout(rate=.1)(x)
    x = Dense(units=50, activation='linear')(x)
    x = Dropout(rate=.1)(x)
    # categorical output of the angle
    angle_out = Dense(units=1, activation='linear', name='angle_out')(x)

    # continous output of throttle
    throttle_out = Dense(units=1, activation='linear', name='throttle_out')(x)

    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])

    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
                  loss={'angle_out': 'mean_squared_error',
                        'throttle_out': 'mean_squared_error'},
                  loss_weights={'angle_out': 0.5, 'throttle_out': .5})
    # TPU
    tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
    strategy = keras_support.TPUDistributionStrategy(tpu_cluster_resolver)
    model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)

    return model

Using TensorFlow backend.


# 学習
学習する前に、ここまでインストールしたライブラリを有効にするために ランタイム-ランタイムの再起動 を実行します。

In [0]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.transform import Lambda


def train(cfg, tub_names, new_model_path, base_model_path=None):
    """
    use the specified data in tub_names to train an artifical neural network
    saves the output trained model as model_name
    """
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    new_model_path = os.path.expanduser(new_model_path)

    kl = KerasLinear()
    if base_model_path is not None:
        base_model_path = os.path.expanduser(base_model_path)
        kl.load(base_model_path)

    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join("/content/mycar/data/", '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                    batch_size=1024,
                                                    train_frac=0.8)

    total_records = len(tubgroup.df)
    total_train = int(total_records * 0.8)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train //1024
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=new_model_path,
             steps=steps_per_epoch,
             train_split=0.8)


if __name__ == '__main__':
    #with tf.device('/gpu:0'):
      cfg = None

      tub = None
      new_model_path = "/content/mycar/models/mypilot"
      base_model_path = None
      cache = None
      train(cfg, tub, new_model_path, base_model_path)

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.45.101.122:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 18005009493283976861)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12103194706472967856)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4077423027071832506)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15075812942443995742)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11294968500045624113)
INFO:tensorflow:*** Available Device: _Dev

In [0]:
!mv /content/keras_tpu.py /content/donkey/donkeycar/parts/